In [1]:
from ecell4 import *
import numpy as np
import math
import sys

run = int(sys.argv[2])
rng = GSLRandomNumberGenerator()
rng.seed(run)
k_factor = float(sys.argv[1])
#rm = 0.005
voxelr = 0.005#*1.0208582  
#r0=0.256318
L = 350*voxelr
D = 1
kd= 4.0*math.pi*D*2*voxelr
ka = k_factor*kd
#ka = (1./r0-1)*6*2**0.5*D*voxelr*kaa/kd
kon = (ka*kd)/(ka+kd)
tm =(2*voxelr)**2/(6*D)
N = 4000
duration =0.05# -math.log(1e-10)/N/kon
dt=tm
w = spatiocyte.SpatiocyteWorld(ones()*L,voxelr)
Vr = w.calculate_volume(ones()*L,voxelr)
lx,ly,lz = tuple(w.calculate_shape(ones()*L,voxelr))
Nv = lx*ly*lz
vv = w.calculate_voxel_volume(voxelr)
phi = 2.*N/Nv
print 'ka,kd,kon,L,dur,N',ka,kd,kon,L,duration,N

with species_attributes():
    A |  {'D': str(0), 'radius':str(voxelr)}
    B |  {'D': str(D), 'radius':str(voxelr)}

with reaction_rules():#for spa and egfrd
    A + B > B | ka

m2 = get_model()

w = spatiocyte.SpatiocyteWorld(ones()*L,voxelr,rng)
w.bind_to(m2)
w.add_molecules(Species('A'), N)
w.add_molecules(Species('B'), N)
obs = FixedIntervalNumberObserver(dt, ('A'))
sim = spatiocyte.SpatiocyteSimulator(w)
sim.initialize()
tlog = dt
tlogs=[]
nlog=[]
while sim.t()<duration:
    if sim.t()>tlog:
        tlogs.append(tlog)
        nlog.append(w.num_particles_exact(Species('A')))
        tlog+=dt
    sim.step()
    #print sim.dt(),tm
    #sim.run(duration, obs)    

name = '/home/chew/runscript/data/keff/kfac'+str(k_factor)+'/run'+str(run)
filename=open(name,'w')
np.savetxt(name,np.column_stack((tlogs,nlog)),delimiter=',',fmt='%s')
filename.close()


L 1.78650185
Vr 5.83415352458
1.25663698739 0.125663706144 0.114239732858
phi 0.103155740411 phir 0.103155740411
